See https://stackoverflow.com/questions/40684543/how-to-make-python-use-ca-certificates-from-mac-os-truststore for a solution to `SSL: CERTIFICATE_VERIFY_FAILED`.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from icalendar import Calendar, Event
import pandas as pd

In [ ]:
# Define url
url = 'https://www.dbukoebenhavn.dk/turneringer_og_resultater/resultatsoegning/programTeam.aspx?TeamId=222951&PoolId=259213&DateFrom=01-01-2018&DateTo=31-12-2018&HomeMatch=1&AwayMatch=1'

In [ ]:
# Read table into dataframe
df = pd.read_html(url, attrs = {'class': 'dbustandard stripes full srDefault srProgram1 srDefaultPadding'})[0]

# Correct column names
new_header = df.columns.to_series().shift(1)
new_header.iat[0] = 'Kampnr'
df = df.rename(new_header, axis='columns')

# Read page into BeautifulSoup (used to get link to match page)
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

# Create calendar object
cal = Calendar()

# Loop through
for row in df.iterrows():
    
    # Create event object
    temp_event = Event()

    # Add title
    event_name = row[1]['Hjemmehold'] + ' - ' + row[1]['Udehold']
    temp_event.add('summary', event_name)

    # Add link to match page
    link = soup.find('a', attrs={'title': 'Kampinformation for {}'.format(row[1]['Kampnr'])})['href']
    url_first_part = '/'.join(url.split('/')[:-1])
    match_url = '/'.join((url_first_part, link))
    temp_event.add('description', match_url)
    
    # Get address from match page
    match_page = urlopen(match_url)
    match_soup = BeautifulSoup(match_page, 'html.parser')
    address = ', '.join(match_soup.find('td', string='Spillested').find_next_sibling().get_text('///').split('///')[1:3])
    temp_event.add('location', address)

    # Add start and end times
    start_time = datetime.strptime(row[1]['Tidspunkt'], '%d-%m-%yKl. %H:%M')
    end_time = start_time + timedelta(hours=2)
    temp_event.add('dtstart', start_time)
    temp_event.add('dtend', end_time)

    # Add event to calendar
    cal.add_component(temp_event)
    
# Write calendar to iCal file
with open('dbu_game_cal.ics', 'wb') as f:
    f.write(cal.to_ical())